# First: Choose best learning rate

### Import

In [ ]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import math

df=read_csv("../input/telugu-6-vowel-dataset/CSV_datasetsix_vowel_dataset_with_class.csv")
le = preprocessing.LabelEncoder()
le.fit(list(df["class"].unique()))
df["class"] = le.transform(df["class"])
print(df.shape)
df.head(1)

In [ ]:
X=df.iloc[:,:df.shape[1]-1]
y=df.iloc[:,df.shape[1]-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
n_inp = X.shape[1]
n_out = len(df["class"].unique())
print(n_inp)
print(n_out)
Ts = [ [1,0,0,0,0,0] , [0,1,0,0,0,0] , [0,0,1,0,0,0] , [0,0,0,1,0,0] ,[0,0,0,0,1,0], [0,0,0,0,0,1]] 

### Sigmoid

In [ ]:
def sig(arr):
    return 1/ (1+np.exp(-np.array(arr)))

### Fit

In [ ]:
#initialize
lrs=[0.0001,0.001,0.005,0.009,0.1,0.101,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

train_errs_list = []
test_errs = []
lr_list = []

In [ ]:
for lr in lrs:
    #train
    #1.initialize
    shape = [n_inp,2,n_out] ; W = [] 
    for w_random in range(len(shape)-1):
        l = np.random.rand(shape[w_random],shape[w_random+1])
        W.append(l)

    train_err = []
    err = 100000
    check_err = False

    for epoch in range(5): #epochs
        if(err < 0.0001 or math.isnan(err) ):
            break
        if(check_err):
            if(np.mean(epoch_err0) == np.mean(epoch_err1)): 
                break
        epoch_err0 = []; epoch_err1 = []

        for i in range(X_train.shape[0]): # batches
            if(math.isnan(err)):
                break
            #2.ForWard
            inp = X_train.iloc[i] ; a = inp ; A = [] ; H = [] ;
            for p in range(len(W)):
                h = np.dot(a,W[p])
                a = sig(h)
                H.append(h)
                A.append(a)

            #3.BackWard
            #3.1.const
            T_y = y_train.iloc[i] 
            T = Ts[T_y]
            O = A[1]
            const = (2 * (T-O)* sig(H[1] ) * (1-sig(H[1])))
            
            #3.2.W[1]
            dev1 = np.dot(const.reshape(-1,1) , (A[0]).reshape(-1,1).T ).T 
            for d in range(len(dev1)):
                W[1][d] = W[1][d] + lr * dev1[d]

            #3.3.W[0]
            WC = np.dot(W[1] , const.reshape(-1,1))
            l = []
            for h in range(len(H[0])):
                l.append((WC[h] * sig(H[0][h]) * (1- sig(H[0][h])) )[0])
            WC_arr = np.array(l).reshape(-1,1)
            ini = np.array(inp).reshape(-1,1)
            W[0] = W[0] + lr * ( np.dot(WC_arr,ini.T) ).T

            #3.4.error
            err = np.sum(T-O)**2/2
            train_err.append(err)

            if(epoch%2 ==0):
                epoch_err0.append(err)
            else:
                epoch_err1.append(err)
                check_err = True
    
    #test
    err=0
    for test in range(X_test.shape[0]): # batches
        inp = X_test.iloc[test] 
        a = inp 
        A = [] ; H = [] 
        for j in range(len(W)):
            h = np.dot(a,W[j])
            a = sig(h)
            H.append(h)
            A.append(a)

        T = y_test.iloc[test] # const
        O = A[1]
        
        Os= list(df["class"].unique())
        O = Os[np.argmax(O)]
        
        if(T!=O):
            err+=1
    print("error at lr: " , lr," : ", err/X_test.shape[0]*100 , " %")
    train_errs_list.append( train_err )
    test_errs.append(err/X_test.shape[0]*100)
    lr_list.append(lr)

### **<span style="color:red"> N.B. high error is due to small number of epochs for time saving as at this very large number of column each iteration take about 15 min -> need very high GPU  </span>**

### Plot best errors

In [ ]:
test_errs_sort = np.sort(test_errs)
train_errs_list_sort = [x for _,x in sorted(zip(test_errs,train_errs_list))]
lr_list_sort = [x for _,x in sorted(zip(test_errs,lr_list))]

In [ ]:
train_errs_list_sort_new = []
# take every 50 points for better view of graph
for i in range(len(train_errs_list_sort)):
    train_errs_list_sort_new.append(train_errs_list_sort[i][0::20])

print(np.shape(train_errs_list_sort[0]))
print(np.shape(train_errs_list_sort_new[0]))

In [ ]:
n=4
fig, ax = plt.subplots(n,2,figsize=(15,15))
color = ['r','b','g','k','y','r','b','g','k','y','r','b','g','k','y','r','b','g','k','y']

for i in range(n):
    ax[i,0].plot(train_errs_list_sort_new[i], color[i]) 
    title = 'lr: '+str(lr_list_sort[i])
    ax[i,0].set_title(title)

for i in range(n,8):
    ax[i-n,1].plot(train_errs_list_sort_new[i], color[i]) 
    title = 'lr: '+str(lr_list_sort[i])
    ax[i-n,1].set_title(title)

### From the graph we can see that lr = 0.0001  is the best and the most stable choice as it is converging while fluctuating

# Now, choose best number of nodes

In [ ]:
lr = 0.0001
train_errs_list = []
test_errs = []
lr_list = []
nodes_list = []
for node in range(1,12):
    #train
    #1.initialize
    shape = [n_inp,node,n_out] ; W = [] 
    for w_random in range(len(shape)-1):
        l = np.random.rand(shape[w_random],shape[w_random+1])
        W.append(l)

    train_err = []
    err = 100000
    check_err = False

    for epoch in range(5): #epochs
        if(err < 0.0001 or math.isnan(err) ):
            break
        if(check_err):
            if(np.mean(epoch_err0) == np.mean(epoch_err1)): 
                break
        epoch_err0 = []; epoch_err1 = []

        for i in range(X_train.shape[0]): # batches
            if(math.isnan(err)):
                break
            #2.ForWard
            inp = X_train.iloc[i] ; a = inp ; A = [] ; H = [] ;
            for p in range(len(W)):
                h = np.dot(a,W[p])
                a = sig(h)
                H.append(h)
                A.append(a)

            #3.BackWard
            #3.1.const
            T_y = y_train.iloc[i] 
            T = Ts[T_y]
            O = A[1]
            const = (2 * (T-O)* sig(H[1] ) * (1-sig(H[1])))
            
            #3.2.W[1]
            dev1 = np.dot(const.reshape(-1,1) , (A[0]).reshape(-1,1).T ).T 
            for d in range(len(dev1)):
                W[1][d] = W[1][d] + lr * dev1[d]

            #3.3.W[0]
            WC = np.dot(W[1] , const.reshape(-1,1))
            l = []
            for h in range(len(H[0])):
                l.append((WC[h] * sig(H[0][h]) * (1- sig(H[0][h])) )[0])
            WC_arr = np.array(l).reshape(-1,1)
            ini = np.array(inp).reshape(-1,1)
            W[0] = W[0] + lr * ( np.dot(WC_arr,ini.T) ).T

            #3.4.error
            err = np.sum(T-O)**2/2
            train_err.append(err)

            if(epoch%2 ==0):
                epoch_err0.append(err)
            else:
                epoch_err1.append(err)
                check_err = True
    
    #test
    err=0
    for test in range(X_test.shape[0]): # batches
        inp = X_test.iloc[test] 
        a = inp 
        A = [] ; H = [] 
        for j in range(len(W)):
            h = np.dot(a,W[j])
            a = sig(h)
            H.append(h)
            A.append(a)

        T = y_test.iloc[test] # const
        O = A[1]
        
        Os= list(df["class"].unique())
        O = Os[np.argmax(O)]
        
        if(T!=O):
            err+=1
    print("error at node: " , node," : ", err/X_test.shape[0]*100 , " %")
    train_errs_list.append( train_err )
    test_errs.append(err/X_test.shape[0]*100)
    nodes_list.append(node)

## plot best error

In [ ]:
test_errs_sort = np.sort(test_errs)
train_errs_list_sort = [x for _,x in sorted(zip(test_errs,train_errs_list))]
nodes_list_sort = [x for _,x in sorted(zip(test_errs,nodes_list))]

In [ ]:
train_errs_list_sort_new = []
# take every 50 points for better view of graph
for i in range(len(train_errs_list_sort)):
    train_errs_list_sort_new.append(train_errs_list_sort[i][0::5])

print(np.shape(train_errs_list_sort))
print(np.shape(train_errs_list_sort_new))

In [ ]:
n=2
fig, ax = plt.subplots(n,2,figsize=(8,8))
color = ['r','b','g','k','y','r','b','g','k','y','r','b','g','k','y','r','b','g','k','y']

for i in range(n):
    ax[i,0].plot(train_errs_list_sort_new[i], color[i]) 
    title = 'node: '+str(nodes_list_sort[i])
    ax[i,0].set_title(title)

for i in range(n,4):
    ax[i-n,1].plot(train_errs_list_sort_new[i], color[i]) 
    title = 'node: '+str(nodes_list_sort[i])
    ax[i-n,1].set_title(title)

## From the graph we can see that nodes = 2 is the best and the most converging choice¶